# Utah Transit Agency Example
In this notebook, we'll predict the energy consumption for a set of trips operated by the Utah Transit Authority (UTA). 

In [1]:
# import logging
import multiprocessing as mp
import os
import time

from nrel.routee.transit import (
    build_routee_features_with_osm,
    predict_for_all_trips,
    repo_root,
)
# Make sure we're specifying paths correctly
os.chdir(repo_root())

# Suppress GDAL/PROJ warnings, which flood the output when we run gradeit
# TODO: resolve underlying issue that generates these warnings
os.environ["PROJ_DEBUG"] = "0"

# # Set up logging: Clear any existing handlers
# logging.getLogger().handlers.clear()

# # Configure basic logging
# logging.basicConfig(
#     level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s - %(message)s"
# )

# logger = logging.getLogger("single_agency_analysis")


In [2]:
# Set inputs
n_proc = mp.cpu_count()
agency = "saltlake"
raster_path = "data/usgs_elevation"
# Number of trips to include in analysis. If None, all will be analyzed.
n_trips_incl = 20

routee_input_df = build_routee_features_with_osm(
    agency=agency,
    n_trips=n_trips_incl,
    add_road_grade=True,
    gradeit_tile_path=raster_path,
    n_processes=n_proc,
)

# logger.info("Finished building RouteE features")
routee_input_df.to_csv(
    f"reports/trip_features/{agency}_trip_features.csv", index=False
)


INFO:gtfs_feature_processing:Feed contains 12037 trips and 89590 shapes
2025-08-07 14:20:13,129 [INFO] - Feed contains 12037 trips and 89590 shapes
INFO:gtfs_feature_processing:Restricted feed to 20 trips and 18 shapes
2025-08-07 14:20:13,132 [INFO] - Restricted feed to 20 trips and 18 shapes
INFO:gtfs_feature_processing:Finished upsampling
2025-08-07 14:20:16,873 [INFO] - Finished upsampling
INFO:gtfs_feature_processing:Original shapes length: 6316
2025-08-07 14:20:16,874 [INFO] - Original shapes length: 6316
INFO:gtfs_feature_processing:Upsampled shapes length: 38496
2025-08-07 14:20:16,877 [INFO] - Upsampled shapes length: 38496
INFO:gtfs_feature_processing:Finished map matching
2025-08-07 14:20:31,710 [INFO] - Finished map matching
INFO:gtfs_feature_processing:Finished attaching timestamps
2025-08-07 14:20:35,608 [INFO] - Finished attaching timestamps
/Users/dmccabe/repos/public/routee-transit/.pixi/envs/dev-py311/lib/python3.11/site-packages/gradeit/grade.py:17: RuntimeWarning: in

In [3]:
# Predict energy consumption
routee_vehicle_model = "Transit_Bus_Battery_Electric"  # RouteE powertrain model
start_time = time.time()
routee_results = predict_for_all_trips(
    routee_input_df=routee_input_df,
    routee_vehicle_model=routee_vehicle_model,
    n_processes=n_proc,
)
routee_results["vehicle"] = routee_vehicle_model
routee_results.to_csv(
    f"reports/energy_predictions/{agency}_link_energy_predictions.csv", index=False
)

# Summarize predictions by trip
agg_cols = [c for c in ["gallons", "kWhs"] if c in routee_results.columns]
energy_by_trip = routee_results.groupby("trip_id").agg(
    {"kilometers": "sum", **{c: "sum" for c in agg_cols}}
)
energy_by_trip["miles"] = 0.6213712 * energy_by_trip["kilometers"]
energy_by_trip["vehicle"] = routee_vehicle_model
# TODO: save geometry data separate from energy predictions to save space
energy_by_trip.drop(columns="kilometers").to_csv(
    f"reports/energy_predictions/{agency}_trip_energy_predictions.csv"
)
# logger.info(f"Finished predictions in {time.time() - start_time:.2f} s")

In [4]:
energy_by_trip.head(10)

,kilometers,kWhs,miles,vehicle
trip_id,,,,
5168740,29.216242,20.059818,18.154132,Transit_Bus_Battery_Electric
5171400,12.255023,8.948623,7.614918,Transit_Bus_Battery_Electric
5171505,12.255023,8.948623,7.614918,Transit_Bus_Battery_Electric
5171928,20.082980,9.536594,12.478985,Transit_Bus_Battery_Electric
5172287,12.808656,6.601645,7.958930,Transit_Bus_Battery_Electric
5173195,12.812999,7.011186,7.961629,Transit_Bus_Battery_Electric
5175541,11.099047,6.022984,6.896628,Transit_Bus_Battery_Electric
5176102,11.856804,9.742285,7.367476,Transit_Bus_Battery_Electric
5176294,7.864189,4.790276,4.886581,Transit_Bus_Battery_Electric
